## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Flatten, MaxPooling1D
from tensorflow.keras.layers import Conv1D, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import time
import numpy as np
import pickle

## Set Memory Growth True

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Loading Training Data

In [3]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [4]:
X=np.reshape(X,(X.shape[0],26,1))

In [5]:
X.shape

(960, 26, 1)

In [6]:
y.shape

(960,)

In [40]:
paths = {"checkpath" : "C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Sentiment/Checkpoints",
         "modelpath" : "C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Sentiment/Checkpoints/Models"}

## Training Model

In [43]:
dense_layers = [1]
conv_layers = [1]
filter_sizes = [32]
layer_sizes = [64]
DOs = [0.2]
epo = 300

for dense_layer in dense_layers:
    for conv_layer in conv_layers:
        for filter_size in filter_sizes:
            for layer_size in layer_sizes:
                for DO in DOs:

                    model = Sequential()

                    model.add(Conv1D(filters=filter_size, kernel_size=3, input_shape=(26,1)))
                    model.add(Activation('relu'))
                    model.add(Dropout(DO))

                    for _ in range(conv_layer-1):
                        model.add(Conv1D(filters=filter_size, kernel_size=3))
                        model.add(Activation('relu'))
                        model.add(Dropout(DO))


                    model.add(Flatten())

                    for _ in range(dense_layer):
                        model.add(Dense(layer_size))
                        model.add(Activation('relu'))
                        model.add(Dropout(DO))

                    model.add(Dense(10))
                    model.add(Activation('softmax'))

                    model.compile(optimizer='adam',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])
                    
                    NAME = f"FEA_EXT_DENSE_{dense_layer}DLay_{conv_layer}CLay_{filter_size}FSz{layer_size}LSz_{DO}DO_{int(time.time())}"
                    
                    filepath=f"{paths['checkpath']}/{NAME}.best.hdf5"
                    #tensorboard = TensorBoard(log_dir=f"C:\\Users\\Lder\\Documents\\ML_PROJ\\log\\fincnn15\\{NAME}")
                    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
                    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
                    
                    

                    hist = model.fit(X, y,
                              batch_size=256,
                              epochs=200,
                              callbacks=[checkpoint],
                              validation_split=0.2)#callbacks=[tensorboard,es,checkpoint]
                    
                    val_acc = hist.history['val_accuracy']
                    max_val_acc = max(val_acc)
                    max_val_acc_loss = hist.history['val_loss'][val_acc.index(max(val_acc))]
                    MODNAME = f"{max_val_acc:.4f}acc_{max_val_acc_loss:.4f}loss"
                    model.load_weights(f"{paths['checkpath']}/{NAME}.best.hdf5")
                    model.save(f"{paths['modelpath']}/{MODNAME}")

Epoch 1/200
3/3 [==============================] - 1s 238ms/step - loss: 2.3156 - accuracy: 0.0978 - val_loss: 2.1506 - val_accuracy: 0.2344

Epoch 00001: val_accuracy improved from -inf to 0.23438, saving model to C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Sentiment/Checkpoints\FEA_EXT_DENSE_1DLay_1CLay_32FSz64LSz_0.2DO_1612866539.best.hdf5
Epoch 2/200
3/3 [==============================] - 0s 28ms/step - loss: 2.0910 - accuracy: 0.2692 - val_loss: 1.9846 - val_accuracy: 0.2865

Epoch 00002: val_accuracy improved from 0.23438 to 0.28646, saving model to C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Sentiment/Checkpoints\FEA_EXT_DENSE_1DLay_1CLay_32FSz64LSz_0.2DO_1612866539.best.hdf5
Epoch 3/200
3/3 [==============================] - 0s 31ms/step - loss: 1.9361 - accuracy: 0.3109 - val_loss: 1.8388 - val_accuracy: 0.3542

Epoch 00003: val_accuracy improved from 0.28646 to 0.35417, saving model to C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Sentiment/Checkpoints\FEA_EXT_DENSE_1DLay_1CLay_32FSz64L


Epoch 00030: val_accuracy did not improve from 0.61458
Epoch 31/200
3/3 [==============================] - 0s 34ms/step - loss: 0.9130 - accuracy: 0.6276 - val_loss: 1.0098 - val_accuracy: 0.6042

Epoch 00031: val_accuracy did not improve from 0.61458
Epoch 32/200
3/3 [==============================] - 0s 33ms/step - loss: 0.9070 - accuracy: 0.6602 - val_loss: 0.9983 - val_accuracy: 0.5938

Epoch 00032: val_accuracy did not improve from 0.61458
Epoch 33/200
3/3 [==============================] - 0s 26ms/step - loss: 0.8785 - accuracy: 0.6813 - val_loss: 0.9900 - val_accuracy: 0.5990

Epoch 00033: val_accuracy did not improve from 0.61458
Epoch 34/200
3/3 [==============================] - 0s 28ms/step - loss: 0.8889 - accuracy: 0.6685 - val_loss: 0.9899 - val_accuracy: 0.5885

Epoch 00034: val_accuracy did not improve from 0.61458
Epoch 35/200
3/3 [==============================] - 0s 27ms/step - loss: 0.8542 - accuracy: 0.6829 - val_loss: 0.9888 - val_accuracy: 0.6146

Epoch 00035: v

Epoch 67/200
3/3 [==============================] - 0s 27ms/step - loss: 0.6187 - accuracy: 0.7713 - val_loss: 0.8691 - val_accuracy: 0.6719

Epoch 00067: val_accuracy improved from 0.65104 to 0.67188, saving model to C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Sentiment/Checkpoints\FEA_EXT_DENSE_1DLay_1CLay_32FSz64LSz_0.2DO_1612866539.best.hdf5
Epoch 68/200
3/3 [==============================] - 0s 27ms/step - loss: 0.6752 - accuracy: 0.7682 - val_loss: 0.8586 - val_accuracy: 0.6667

Epoch 00068: val_accuracy did not improve from 0.67188
Epoch 69/200
3/3 [==============================] - 0s 28ms/step - loss: 0.6319 - accuracy: 0.7648 - val_loss: 0.8536 - val_accuracy: 0.6667

Epoch 00069: val_accuracy did not improve from 0.67188
Epoch 70/200
3/3 [==============================] - 0s 25ms/step - loss: 0.5971 - accuracy: 0.7879 - val_loss: 0.8616 - val_accuracy: 0.6771

Epoch 00070: val_accuracy improved from 0.67188 to 0.67708, saving model to C:/Users/Lder/Documents/MAJOR_PROJ/Audio_Se

3/3 [==============================] - 0s 32ms/step - loss: 0.5080 - accuracy: 0.8130 - val_loss: 0.8097 - val_accuracy: 0.7031

Epoch 00104: val_accuracy did not improve from 0.70833
Epoch 105/200
3/3 [==============================] - 0s 29ms/step - loss: 0.4503 - accuracy: 0.8405 - val_loss: 0.8106 - val_accuracy: 0.6927

Epoch 00105: val_accuracy did not improve from 0.70833
Epoch 106/200
3/3 [==============================] - 0s 29ms/step - loss: 0.4708 - accuracy: 0.8203 - val_loss: 0.8086 - val_accuracy: 0.6927

Epoch 00106: val_accuracy did not improve from 0.70833
Epoch 107/200
3/3 [==============================] - 0s 27ms/step - loss: 0.4631 - accuracy: 0.8387 - val_loss: 0.8117 - val_accuracy: 0.6927

Epoch 00107: val_accuracy did not improve from 0.70833
Epoch 108/200
3/3 [==============================] - 0s 28ms/step - loss: 0.4550 - accuracy: 0.8444 - val_loss: 0.8103 - val_accuracy: 0.6927

Epoch 00108: val_accuracy did not improve from 0.70833
Epoch 109/200
3/3 [=====

3/3 [==============================] - 0s 27ms/step - loss: 0.3522 - accuracy: 0.8752 - val_loss: 0.8004 - val_accuracy: 0.7240

Epoch 00142: val_accuracy did not improve from 0.72917
Epoch 143/200
3/3 [==============================] - 0s 31ms/step - loss: 0.3615 - accuracy: 0.8765 - val_loss: 0.8015 - val_accuracy: 0.7083

Epoch 00143: val_accuracy did not improve from 0.72917
Epoch 144/200
3/3 [==============================] - 0s 37ms/step - loss: 0.3590 - accuracy: 0.8747 - val_loss: 0.7967 - val_accuracy: 0.7031

Epoch 00144: val_accuracy did not improve from 0.72917
Epoch 145/200
3/3 [==============================] - 0s 35ms/step - loss: 0.3585 - accuracy: 0.8628 - val_loss: 0.7902 - val_accuracy: 0.7031

Epoch 00145: val_accuracy did not improve from 0.72917
Epoch 146/200
3/3 [==============================] - 0s 35ms/step - loss: 0.3403 - accuracy: 0.8950 - val_loss: 0.7843 - val_accuracy: 0.7083

Epoch 00146: val_accuracy did not improve from 0.72917
Epoch 147/200
3/3 [=====

Epoch 181/200
3/3 [==============================] - 0s 27ms/step - loss: 0.2800 - accuracy: 0.9058 - val_loss: 0.7606 - val_accuracy: 0.7344

Epoch 00181: val_accuracy did not improve from 0.75521
Epoch 182/200
3/3 [==============================] - 0s 30ms/step - loss: 0.2934 - accuracy: 0.8910 - val_loss: 0.7643 - val_accuracy: 0.7396

Epoch 00182: val_accuracy did not improve from 0.75521
Epoch 183/200
3/3 [==============================] - 0s 30ms/step - loss: 0.3286 - accuracy: 0.8958 - val_loss: 0.7669 - val_accuracy: 0.7292

Epoch 00183: val_accuracy did not improve from 0.75521
Epoch 184/200
3/3 [==============================] - 0s 29ms/step - loss: 0.2699 - accuracy: 0.9178 - val_loss: 0.7616 - val_accuracy: 0.7344

Epoch 00184: val_accuracy did not improve from 0.75521
Epoch 185/200
3/3 [==============================] - 0s 30ms/step - loss: 0.2952 - accuracy: 0.9006 - val_loss: 0.7555 - val_accuracy: 0.7500

Epoch 00185: val_accuracy did not improve from 0.75521
Epoch 186/

In [47]:
model = tf.keras.models.load_model(f"{paths['modelpath']}/0.7760acc_0.7530loss")

In [48]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 24, 32)            128       
_________________________________________________________________
activation_45 (Activation)   (None, 24, 32)            0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 24, 32)            0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 768)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                49216     
_________________________________________________________________
activation_46 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 64)              